In [1]:
%%configure -f
{ "conf":{
          "spark.pyspark.python": "python3",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type":"native",
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
         }
}

In [2]:
sc.install_pypi_package("pandas")
sc.install_pypi_package("numpy")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1747168862658_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.1
    Not uninstalling python-dateutil at /usr/lib/python3.9/site-packages, outside environment /mnt1/yarn/usercache/livy/appcache/application_1747168862658_0001/container_1747168862658_0001_01_000001/tmp/spark-5846d2fa-57cb-4dd9-88b3-1a8b139f971a
    Can't uninstall 'python-dateutil'. No files were found to uninstall.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 2.23.11 requires python-dateutil<=2.9.0,>=2.1, but you have python-dateutil 2.9.0.post0 which is incompatible.


In [3]:
import pandas as pd
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode, collect_list
from pyspark.ml.feature import HashingTF, IDF
from pyspark.sql.functions import lower, regexp_replace, split
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.functions import concat_ws
from pyspark.sql.functions import expr
from pyspark.sql.functions import trim, col
from pyspark.sql.functions import filter as array_filter
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col, udf, desc
from pyspark.sql.types import DoubleType
import numpy as np
import time

spark = SparkSession.builder.appName("Movies").getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
start = time.perf_counter()
movies_cleaned = (
    spark.read
         .option("header",    "true")
         .option("inferSchema","true")
         .csv("s3://recomendandoando/processed/movies_filtered/")
)
end = time.perf_counter()
print(f" Movies_filtered loaded in {end - start:.2f} s")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

 Movies_filtered loaded in 8.98 s

In [13]:
# Transformar el texto para vectorizarlo
start = time.perf_counter()
movies_cleaned = movies_cleaned.withColumn("combined_text", concat_ws(" ","title", "genres"))
# Texto en minusculas y eliminar caracteres especiales
movies_cleaned = movies_cleaned.withColumn("processed_text", lower(col("combined_text")))
movies_cleaned = movies_cleaned.withColumn("processed_text", regexp_replace(col("processed_text"), "[^a-zA-Z\\s]", " "))

# Convertir la columna 'processed_text' en un array de palabras
movies_cleaned = movies_cleaned.withColumn("words", split(col("processed_text"), " "))

# Eliminar stopwords
remover = StopWordsRemover(inputCol="words", outputCol="filtered_text")
movies_cleaned = remover.transform(movies_cleaned)
movies_cleaned = movies_cleaned.withColumn(
    "filter_text_clean",
    array_filter(col("filtered_text"), lambda x: trim(x) != "")
)


movies_cleaned.select("filter_text_clean").show(truncate=False)
end = time.perf_counter()
print(f"Clean time: {end - start:.2f} s")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------------------------------------------+
|filter_text_clean                                            |
+-------------------------------------------------------------+
|[toy, story, adventure, animation, children, comedy, fantasy]|
|[jumanji, adventure, children, fantasy]                      |
|[grumpier, old, men, comedy, romance]                        |
|[waiting, exhale, comedy, drama, romance]                    |
|[father, bride, part, ii, comedy]                            |
|[heat, action, crime, thriller]                              |
|[sabrina, comedy, romance]                                   |
|[tom, huck, adventure, children]                             |
|[sudden, death, action]                                      |
|[goldeneye, action, adventure, thriller]                     |
|[american, president, comedy, drama, romance]                |
|[dracula, dead, loving, comedy, horror]                      |
|[balto, adventure, animation, children]

In [14]:
# Crear un HashingTF para vectorizar el texto procesado
start = time.perf_counter()
hashingTF = HashingTF(inputCol="filtered_text", outputCol="raw_features", numFeatures=20)

# Calcular TF-IDF
idf = IDF(inputCol="raw_features", outputCol="features")

# Pipeline para las transformaciones
pipeline = Pipeline(stages=[hashingTF, idf])

model = pipeline.fit(movies_cleaned)
movies_cleaned = model.transform(movies_cleaned)
end = time.perf_counter()
print(f"Hashing and TD-IDF execution time: {end - start:.2f} s")


movies_cleaned.select("movieId", "features").show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Hashing and TD-IDF execution time: 3.92 s
+-------+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|movieId|features                                                                                                                                                        |
+-------+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1      |(20,[1,12,13,15,17,19],[1.467113497528166,0.00954862220713434,2.5735505396155305,2.0569516086221125,2.0988171224776484,1.5756070940813949])                     |
|2      |(20,[1,12,13,16,19],[1.467113497528166,0.008355044431242548,1.2867752698077652,2.225940585204374,1.5756070940813949])                                           |
|3      |(20,[3,5,6,7,12],[2.1383185373871587,1.8024025903290017,0.9468912533247323,2.3463697406556587,

In [15]:
# Función UDF para calcular la similitud de coseno
def cosine_similarity(vec1, vec2):
    # Convertir vectores de SparseVector a arrays
    vec1_array = np.array(vec1.toArray())
    vec2_array = np.array(vec2.toArray())

    # Calcular el producto punto y las longitudes de los vectores
    dot_product = np.dot(vec1_array, vec2_array)
    norm_a = np.linalg.norm(vec1_array)
    norm_b = np.linalg.norm(vec2_array)

    # Calcular la similitud de coseno
    return float(dot_product / (norm_a * norm_b)) if norm_a and norm_b else 0.0

# Registrar la UDF de similitud de coseno
cosine_udf = udf(cosine_similarity, DoubleType())
start = time.perf_counter()

# matriz de similitudes
cosine_sim_matrix = movies_cleaned.alias("df1").join(movies_cleaned.alias("df2"), col("df1.movieId") != col("df2.movieId")) \
    .withColumn("cosine_sim", cosine_udf(col("df1.features"), col("df2.features")))
end = time.perf_counter()
print(f"Similarity matrix execution time: {end - start:.2f} s")


cosine_sim_matrix.select("df1.movieId", "df2.movieId", "cosine_sim").show(truncate=False)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Similarity matrix execution time: 0.15 s
+-------+-------+---------------------+
|movieId|movieId|cosine_sim           |
+-------+-------+---------------------+
|1      |2      |0.531283421719517    |
|1      |3      |5.420238182686733E-6 |
|1      |4      |8.83201872225992E-6  |
|1      |5      |4.647668127934219E-6 |
|1      |6      |0.2310918019925448   |
|1      |7      |2.4285195286823596E-5|
|1      |8      |0.7417179243571477   |
|1      |9      |0.22266185057699128  |
|1      |10     |0.560381729705262    |
|1      |11     |1.2565158983679533E-5|
|1      |12     |0.24171866428401453  |
|1      |13     |0.5411721076019801   |
|1      |14     |9.955230591563787E-6 |
|1      |15     |0.6101948157733894   |
|1      |16     |1.2186375080422272E-5|
|1      |17     |5.76061651572219E-6  |
|1      |18     |9.032851969707335E-6 |
|1      |19     |4.269840453332562E-6 |
|1      |20     |0.4568134142566331   |
|1      |21     |6.494547704413935E-6 |
+-------+-------+---------------------+

In [16]:
# Función para obtener recomendaciones basadas en la similitud de coseno
def get_recommendations(movie_name, cosine_sim_matrix, df, top_n=10):
    # Buscar el movie_id basado en el nombre
    movie_id_row = df.filter(col("title") == movie_name).select("movieId").first()

    if movie_id_row is None:
        return "Movie not found"

    movie_id = movie_id_row["movieId"]

    # Filtrar las similitudes para el movie_id dado
    recommendations = cosine_sim_matrix.filter(col("df1.movieId") == movie_id) \
        .orderBy(desc("cosine_sim")) \
        .select("df2.movieId", "df2.title", "cosine_sim")

    # Mostrar las top_n recomendaciones
    recommendations = recommendations.limit(top_n)

    return recommendations

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
recommended_movies = get_recommendations(movie_name="Toy Story (1995)", cosine_sim_matrix=cosine_sim_matrix, df=movies_cleaned, top_n=20)
recommended_movies.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------------------------------------------+------------------+
|movieId|title                                                  |cosine_sim        |
+-------+-------------------------------------------------------+------------------+
|3114   |Toy Story 2 (1999)                                     |0.9999998567812832|
|235225 |Once Upon a Snowman (2020)                             |0.9622904605790515|
|164576 |Time Raiders (2016)                                    |0.9443553405647069|
|249028 |One Piece: Curse of the Sacred Sword (2004)            |0.9428090874299441|
|2987   |Who Framed Roger Rabbit? (1988)                        |0.9356978045402591|
|255927 |Goat Story With Cheese (2012)                          |0.9355285118023223|
|2355   |Bug's Life, A (1998)                                   |0.9355284735301628|
|201588 |Toy Story 4 (2019)                                     |0.9355283587139129|
|145082 |The Snow Queen 2: Refreeze (2014)                      |

In [18]:
from pyspark.ml import PipelineModel

# `model` is the fitted Pipeline from your code above
pipeline_path = "s3://recomendandoando/models/tfidf_content_pipeline"
recommended_movies.write \
    .mode("overwrite") \
    .parquet("s3://recomendandoando/recommendations/content_based/parquet/")
model.write().overwrite().save(pipeline_path)
print(f"Pipeline saved to {pipeline_path}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Pipeline saved to s3://recomendandoando/models/tfidf_content_pipeline

In [19]:
# For later implementation with new movies
## load recs
#recs = spark.read.parquet("s3://recomendandoando/recommendations/content_based/parquet/")
#recs.show()

## load pipeline and re-generate features if you ingest new movies
#pipeline = PipelineModel.load("s3://recomendandoando/models/tfidf_content_pipeline")
#new_movies = spark.read.csv("s3://...")                # however you ingest more titles
#new_featurized = pipeline.transform(new_movies)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…